In [1]:
import os
import cv2
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

In [2]:
# === GPU ===
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cpu


In [3]:
# === Параметры ===
video_dir = "dataset2"
frame_size = (112, 112)
frames_per_video = 16
batch_size = 4
epochs = 10

# === Преобразование кадров ===
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize(frame_size),
    transforms.ToTensor()
])

# === Извлечение кадров из видео ===
def extract_frames(video_path, num_frames=16):
    cap = cv2.VideoCapture(video_path)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    frame_indices = torch.linspace(0, total_frames - 1, num_frames).long()
    frames = []
    
    for i in range(total_frames):
        ret, frame = cap.read()
        if not ret:
            break
        if i in frame_indices:
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            frame_tensor = transform(frame)
            frames.append(frame_tensor)
    
    cap.release()

    # Если кадров меньше — дублируем последний
    if len(frames) < num_frames:
        for _ in range(num_frames - len(frames)):
            frames.append(frames[-1])
    
    return torch.stack(frames)  # (T, C, H, W)

# === Получение всех видео и меток ===
all_samples = []
for label, folder in enumerate(["NonViolence", "Violence"]):
    folder_path = os.path.join(video_dir, folder)
    for file in os.listdir(folder_path):
        if file.endswith(".mp4"):
            all_samples.append((os.path.join(folder_path, file), label))

# === Разделение на train/test ===
train_samples, test_samples = train_test_split(
    all_samples,
    test_size=0.2,
    stratify=[label for _, label in all_samples],
    random_state=42
)

In [4]:
# === Dataset ===
class ViolenceDataset(Dataset):
    def __init__(self, samples):
        self.samples = samples

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        video_path, label = self.samples[idx]
        video_tensor = extract_frames(video_path, num_frames=frames_per_video)
        return video_tensor, torch.tensor(label, dtype=torch.long)

# === DataLoader ===
train_dataset = ViolenceDataset(train_samples)
test_dataset = ViolenceDataset(test_samples)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


In [5]:
# === Модель 3D CNN ===
class Simple3DCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            nn.Conv3d(3, 32, kernel_size=(3,3,3), padding=1),
            nn.ReLU(),
            nn.MaxPool3d((1,2,2)),

            nn.Conv3d(32, 64, kernel_size=(3,3,3), padding=1),
            nn.ReLU(),
            nn.MaxPool3d((2,2,2)),

            nn.Flatten(),
            nn.Linear(64 * 8 * 28 * 28, 128),
            nn.ReLU(),
            nn.Linear(128, 2)
        )

    def forward(self, x):
        # x: (B, T, C, H, W) -> (B, C, T, H, W)
        x = x.permute(0, 2, 1, 3, 4)
        return self.model(x)

model = Simple3DCNN().to(device)


In [6]:
# === Обучение ===
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

def evaluate(model, dataloader):
    model.eval()
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            preds = outputs.argmax(dim=1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    acc = accuracy_score(all_labels, all_preds)
    return acc, all_labels, all_preds

for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    all_preds = []
    all_labels = []
    for inputs, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}"):
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)

        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        all_preds.extend(outputs.argmax(dim=1).cpu().numpy())
        all_labels.extend(labels.cpu().numpy())
    
    acc_train = accuracy_score(all_labels, all_preds)
    acc_test, test_true, test_pred = evaluate(model, test_loader)
    
    print(f"[Epoch {epoch+1}] Loss: {running_loss:.4f} | Train Acc: {acc_train:.4f} | Test Acc: {acc_test:.4f}")

Epoch 1: 100%|██████████| 762/762 [11:25<00:00,  1.11it/s]  


[Epoch 1] Loss: 392.5915 | Train Acc: 0.7383 | Test Acc: 0.8215


Epoch 2: 100%|██████████| 762/762 [11:47<00:00,  1.08it/s]


[Epoch 2] Loss: 184.6781 | Train Acc: 0.9012 | Test Acc: 0.8753


Epoch 3: 100%|██████████| 762/762 [11:30<00:00,  1.10it/s]


...

[Epoch 8] Loss: 13.4695 | Train Acc: 0.9944 | Test Acc: 0.9291


Epoch 9: 100%|██████████| 762/762 [11:28<00:00,  1.11it/s]  


[Epoch 9] Loss: 19.0764 | Train Acc: 0.9908 | Test Acc: 0.9436


Epoch 10: 100%|██████████| 762/762 [09:31<00:00,  1.33it/s]


[Epoch 10] Loss: 10.2405 | Train Acc: 0.9964 | Test Acc: 0.9423


In [7]:
# === Метрики на тесте ===
print("\n📊 Классификационный отчёт (Test Set):")
print(classification_report(test_true, test_pred, target_names=["NonViolence", "Violence"]))

# === Сохранение модели ===
torch.save(model.state_dict(), "violence_detector.pth")
print("✅ Модель сохранена как 'violence_detector.pth'")


📊 Классификационный отчёт (Test Set):
              precision    recall  f1-score   support

 NonViolence       0.94      0.95      0.94       372
    Violence       0.95      0.94      0.94       390

    accuracy                           0.94       762
   macro avg       0.94      0.94      0.94       762
weighted avg       0.94      0.94      0.94       762

✅ Модель сохранена как 'violence_detector.pth'
